In [4]:
import pandas as pd
import xarray as xr
import glob

In [ ]:
tas_df = pd.read_csv('../results/tas/benchmark_results.csv')
tos_df = pd.read_csv('../results/tos/benchmark_results.csv')
pr_df = pd.read_csv('../results/pr/benchmark_results.csv')
clt_df = pd.read_csv('../results/clt/benchmark_results.csv')
od550aer_df = pd.read_csv('../results/od550aer/benchmark_results.csv')

df = pd.concat([tas_df,tos_df,pr_df,clt_df,od550aer_df],axis=0)

df['region'] = 'global'
df.loc[(df['lat_min'] == -90) & (df['lat_max'] == 0), 'region'] = 'southern_hemisphere'
df.loc[(df['lat_min'] == 0) & (df['lat_max'] == 90), 'region'] = 'northern_hemisphere'
df.loc[(df['lat_min'] == -30) & (df['lat_max'] == 30), 'region'] = 'tropics'

df['period'] = 'Historical (2005-2014)'
df.loc[(df['start_year'] == 2015) & (df['end_year'] == 2024), 'period'] = 'SSP2-4.5'
df.loc[(df['start_year'] == 1990) & (df['end_year'] == 2014), 'period'] = 'Historical (1990-2014)'
df.loc[(df['start_year'] == 1960) & (df['end_year'] == 2014), 'period'] = 'Historical (1960-2014)'

df_piv = df.pivot_table(index=['model','variable','metric','region'],columns='period',values='value').reset_index()

df_piv['Change (hist 2005)'] = df_piv['SSP2-4.5'] - df_piv['Historical (2005-2014)']
df_piv['Percent Change (hist 2005)'] = (df_piv['SSP2-4.5'] - df_piv['Historical (2005-2014)'])*100/df_piv['Historical (2005-2014)']

# df_piv.to_csv('../results/processed_results/benchmark_results.csv')

In [ ]:
for metric in ['spatial_mae','spatial_rmse']:
    var_df_list = []

    for var in ['tos','tas','pr','clt','od550aer']:

        files = glob.glob(f'../backend/results/{var}/*')
        files = [file for file in files if metric in file]

        df_list = []

        for file in files:

            ds = xr.open_dataset(file)
            df = ds.to_dataframe()
            model = file.split('/')[-1].split('_')[0]
            lat_min = file.split('/')[-1].split('_')[3]
            lat_max = file.split('/')[-1].split('_')[4]
            if lat_min == '-90' and lat_max == '90':
                region = 'global'
            elif lat_min == '-90' and lat_max == '0':
                region = 'southern_hemisphere'
            elif lat_min == '0' and lat_max == '90':
                region = 'northern_hemisphere'
            elif lat_min == '-30' and lat_max == '30':
                region = 'tropics'

            df['model'] = model
            df['region'] = region
            df['variable'] = var
            df_list.append(df)

        var_df_list.append(pd.concat(df_list).reset_index().pivot(index=['time','model','variable'],columns='region',values=metric))

    pd.concat(var_df_list).reset_index().to_csv(f'../results/processed_results/{metric}_results.csv')

In [ ]:
df_mae = pd.read_csv('../results/processed_results/spatial_mae_results.csv').drop(columns=['Unnamed: 0'])
df_rmse = pd.read_csv('../results/processed_results/spatial_rmse_results.csv').drop(columns=['Unnamed: 0'])
df_mae['metric'] = "MAE"
df_rmse['metric'] = "RMSE"
df = pd.concat([df_mae,df_rmse])
df.to_csv('../results/processed_results/benchmark_results_time_series.csv')